# Import Libraries

In [1]:
import tensorflow as tf
assert float(tf.__version__[:3]) >= 2.3

import os
import numpy as np
import glob
import time
import matplotlib.pyplot as plt
import time
from tensorflow.keras.preprocessing import image

# from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

# Load dataset

In [2]:
class_names = ['construction_danger', 'construction_safe', 'park_danger', 'park_safe', 'road_danger', 'road_safe']
label_list = []
image_list = []
dataset_list = []

for label in class_names:
    img_path_list = glob.glob(f'../dataset/{label}/*')
    # print(img_path_list)
    
    for i in range(len(img_path_list)):
        image = tf.io.read_file(img_path_list[i])
        image = tf.io.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [224, 224])
        dataset_list.append(tf.expand_dims(image, 0))
        image_list.append(image)
        label_list.append(label)

# compiled model test

In [3]:
class Saved_model_test:
    def __init__(self, saved_model_dir, model_type, class_names):
        self.saved_model_dir = saved_model_dir
        
        start_time = time.time()
        self.saved_model_loaded = tf.keras.models.load_model(saved_model_dir)
        end_time = time.time()
        term = end_time - start_time
        self.loading_term = term
        
        self.class_names = class_names
        self.model_type = model_type

        if self.model_type == 'keras':
            self.infer = self.saved_model_loaded.predict
            self.output_layer = None
        elif self.model_type == 'saved_model':
            self.infer = self.saved_model_loaded.signatures['serving_default']
            self.output_layer = next(iter(self.infer.structured_outputs.keys()))
        
        self.saved_model_infer = {}
        self.infer_result = []
        
    
    def check_speed(self, input):
        N_warmup_run = 50
        N_run = 20
        elapsed_time = []
        
        for i in range(N_warmup_run):
            preds = self.infer(input)
        
        for i in range(N_run):
            start_time = time.time()
            for j in range(N_warmup_run):
                preds = self.infer(input)
            end_time = time.time()
            elapsed_time = np.append(elapsed_time, end_time - start_time)
        self.elapsed_time = elapsed_time
        
        total = 0
        for i in range(N_run):
            total += elapsed_time[i]
        # total /= len(elapsed_time)
        self.speed = (N_run * N_warmup_run)/total
        
        
    def test(self, dataset_list, label_list):
        
        result_list = []
        for i in range(len(dataset_list)):
            if self.model_type == 'keras':
                preds = self.infer(dataset_list[i])    
            elif self.model_type == 'saved_model':
                preds =  self.infer(dataset_list[i])[self.output_layer].numpy()
            result_list.append([label_list[i], self.class_names[np.argmax(preds[0], axis=0)],  preds])
            
        self.result_list = result_list        
    
    def debug(self, dataset_list, label_list):
        
        result_list = []
        np_label = np.array(label_list)
        for label in self.class_names:
            first_idx = np.where(np_label == label)[0][0]
            last_idx = np.where(np_label == label)[0][-1]
            label_len = last_idx - first_idx + 1
            
            fig = plt.figure(figsize=(50, 50))
            
            
            for i in range(label_len):
                fig.add_subplot(10, 10, i+1)
                
                if self.model_type == 'keras':
                    preds = self.infer(dataset_list[first_idx+i])    
                elif self.model_type == 'saved_model':
                    preds =  self.infer(dataset_list[first_idx+i])[self.output_layer].numpy()
                _label = self.class_names[np.argmax(preds[0], axis=0)]
                result_list.append([label_list[first_idx+i], _label,  preds])

                plt.title(f'{label_list[first_idx+i]}|{_label}')
                plt.imshow(image_list[first_idx+i]/256.)
            # plt.show()
            plt.savefig(f'{self.saved_model_dir}_{label}.png', dpi=300)
        self.result_list = result_list
    
        
    def calc_accuracy(self):
        self.accuracy = {}
        
        c = np.array(self.result_list)
        for label in self.class_names:
            first_idx = np.where(c[0:, 0:1] == label)[0][0]
            last_idx = np.where(c[0:, 0:1] == label)[0][-1]

            label_len = len(np.where(c[first_idx:last_idx+1, 0:1] == label)[0])
            correct = len(np.where(c[first_idx:last_idx+1:, 1:2] == label)[0])
            self.accuracy[label] = correct/label_len

In [4]:
def debug(input, type, class_names):
    model_test = Saved_model_test(input, type, class_names)
    print(f'loading term (sec): {model_test.loading_term}')
    
    model_test.check_speed(dataset_list[0])
    print(f'images per sec: {model_test.speed}')

    model_test.debug(dataset_list, label_list)
    model_test.calc_accuracy()
    print(f'accuracy: {model_test.accuracy}')
    
def test(input, type, class_names):
    model_test = Saved_model_test(input, type, class_names)
    print(f'loading term (sec): {model_test.loading_term}')
    
    model_test.check_speed(dataset_list[0])
    print(f'images per sec: {model_test.speed}')

    model_test.test(dataset_list, label_list)
    model_test.calc_accuracy()
    print(f'accuracy: {model_test.accuracy}')

# test

## custom_saved_model

In [ ]:
# custom_saved_model
debug('../saved_models/custom_saved_model', 'keras', class_names)

In [ ]:
# custom_resnet_saved_model
debug('../saved_models/custom_resnet_saved_model', 'keras', class_names)

In [ ]:
# custom_resnet_saved_model_TFTRT_F32
debug('custom_resnet_saved_model_TFTRT_F32', 'saved_model', class_names)